# Import libraries

In [1]:
import nltk, re, pprint
from nltk import word_tokenize
import gc

# Read data

In [2]:
import json
DATA_PATH = '/kaggle/input/vietnamese-online-news-dataset/news_dataset.json'

with open(DATA_PATH, 'r') as f:
    raw = json.load(f)

In [3]:
raw[0]

{'id': 218270,
 'author': '',
 'content': "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường

In [4]:
len(raw)

184539

In [5]:
raw_content = []
for item in raw:
    raw_content.append(item['content'])
raw_content[0]

"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn

# Preprocess data

In [6]:
# function to normalize text
def normalize_text(text):
    # remove html
    clean_text = re.sub(r'<[^>]*>', '', text)
    
    # remove ads
    # method 1
    clean_text = clean_text.replace('(adsbygoogle = window.adsbygoogle || []).push({});', '')
    # method 2
    # cleaned_text = re.sub(r'\(.*?\|\|\s*\[\s*\].*?\{\s*\}\s*\);', '', clean_text)
    
    # remove http
    clean_text = re.sub(r'Nguồn:\s*http?:\/\/\S+', '', clean_text)
    clean_text = re.sub(r'http\S*', '', clean_text)
    
    # lowercase
    clean_text = clean_text.lower()
    
    # remove punctuation
    clean_text = re.sub(r'[^\w\s/]', ' ', clean_text)
    
    # remove white space
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

In [7]:
clean_content = []
for item in raw_content:
    clean_content.append(normalize_text(item))
clean_content[0]

'chiều 31/7 công an tỉnh thừa thiên huế đã có thông tin ban đầu về vụ nổ súng cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo tp huế tỉnh thừa thiên huế thông sài gòn giải phóng khoảng 12h30 ngày 31/7 một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi phía trước chợ đông ba rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng sau đó đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội cách khu vực gây án khoảng 300m giám đốc công an tỉnh thừa thiên huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ khoanh vùng và ngăn không để người dân đi vào hiện trường hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường đóng cửa nhà đề phòng đạn lạc tuy nhiên thấy vàng bị ném ra đường nhiều người đua nhau nhặt tạo cảnh nhốn nháo trước cổng chợ do đây

In [8]:
!pip install -q underthesea

In [9]:
# tokenize Vietnamese text
from underthesea import word_tokenize
tokens_list = []
for item in clean_content:
    tokens = word_tokenize(item, format="text").split()
    tokens_list.append(tokens)
tokens_list[0]

['chiều',
 '31/7',
 'công_an',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'đã',
 'có',
 'thông_tin',
 'ban_đầu',
 'về',
 'vụ',
 'nổ_súng',
 'cướp',
 'tiệm',
 'vàng',
 'tại',
 'chợ',
 'đông',
 'ba',
 'nằm',
 'trên',
 'đường',
 'trần_hưng_đạo',
 'tp',
 'huế',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'thông_sài',
 'gòn',
 'giải_phóng',
 'khoảng',
 '12',
 'h30',
 'ngày',
 '31/7',
 'một',
 'đối_tượng',
 'sử_dụng',
 'súng',
 'ak',
 'bất_ngờ',
 'xông',
 'vào',
 'tiệm',
 'vàng',
 'hoàng_đức',
 'và',
 'thái_lợi',
 'phía',
 'trước',
 'chợ',
 'đông',
 'ba',
 'rồi',
 'nổ_súng',
 'chỉ_thiên',
 'liên_tiếp',
 'uy_hiếp',
 'chủ',
 'tiệm',
 'để',
 'cướp',
 'vàng',
 'sau',
 'đó',
 'đối_tượng',
 'mang',
 'số',
 'vàng',
 'vừa',
 'cướp',
 'được',
 'vứt',
 'ra',
 'vỉa_hè',
 'rồi',
 'đi',
 'bộ',
 'đến',
 'khu_vực',
 'cầu',
 'gia_hội',
 'cách',
 'khu_vực',
 'gây',
 'án',
 'khoảng',
 '300',
 'm',
 'giám_đốc',
 'công_an',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'lập_tức',
 'trực_tiếp',
 'chỉ_đạo',
 'các',
 'lực_lượng',
 'chức_năng',
 'g

In [10]:
# # tokenize text (word tokenize)
# tokens_list = []
# for item in clean_content:
#     tokens = word_tokenize(item)
#     tokens_list.append(tokens)
# tokens_list[0]

In [11]:
# remove number from tokenize list
for i in range(len(tokens_list)):
    tokens_list[i] = [item for item in tokens_list[i] if not item.isdigit()]

In [12]:
# build stop words list bases on frequency
from collections import Counter
stop_words = []

all_words = [item for sublist in tokens_list for item in sublist]
word_count = Counter(all_words)

# take a list of most common 50 words
top_50_words = word_count.most_common(50)

for word, count in top_50_words:
    stop_words.append(word)
    print(f'{word}: {count}')

và: 1215809
của: 1059891
các: 872034
trong: 779686
là: 774537
có: 766143
được: 676238
cho: 647005
với: 638984
đã: 597587
không: 520356
người: 492074
một: 453266
để: 453071
khi: 439973
những: 426017
ở: 394306
đến: 385399
này: 384128
về: 366815
từ: 362591
đó: 348388
năm: 344861
nhiều: 338405
cũng: 331990
vào: 320259
trên: 317716
tại: 317270
sẽ: 308692
theo: 288355
ra: 277447
ngày: 257959
sau: 248396
ông: 239582
hơn: 235520
như: 221776
phải: 215828
anh: 212596
việc: 206784
nhưng: 203225
có_thể: 200428
lại: 197961
bị: 190958
đang: 188904
làm: 187422
còn: 185304
biết: 183121
tôi: 181574
việt_nam: 181023
trước: 178699


In [13]:
stop_words.append('/')
stop_words = set(stop_words)
# remove stop words
for i in range(len(tokens_list)):
    tokens_list[i] = [item for item in tokens_list[i] if item not in stop_words]

In [14]:
tokens_list[0]

['chiều',
 '31/7',
 'công_an',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'thông_tin',
 'ban_đầu',
 'vụ',
 'nổ_súng',
 'cướp',
 'tiệm',
 'vàng',
 'chợ',
 'đông',
 'ba',
 'nằm',
 'đường',
 'trần_hưng_đạo',
 'tp',
 'huế',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'thông_sài',
 'gòn',
 'giải_phóng',
 'khoảng',
 'h30',
 '31/7',
 'đối_tượng',
 'sử_dụng',
 'súng',
 'ak',
 'bất_ngờ',
 'xông',
 'tiệm',
 'vàng',
 'hoàng_đức',
 'thái_lợi',
 'phía',
 'chợ',
 'đông',
 'ba',
 'rồi',
 'nổ_súng',
 'chỉ_thiên',
 'liên_tiếp',
 'uy_hiếp',
 'chủ',
 'tiệm',
 'cướp',
 'vàng',
 'đối_tượng',
 'mang',
 'số',
 'vàng',
 'vừa',
 'cướp',
 'vứt',
 'vỉa_hè',
 'rồi',
 'đi',
 'bộ',
 'khu_vực',
 'cầu',
 'gia_hội',
 'cách',
 'khu_vực',
 'gây',
 'án',
 'khoảng',
 'm',
 'giám_đốc',
 'công_an',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'lập_tức',
 'trực_tiếp',
 'chỉ_đạo',
 'lực_lượng',
 'chức_năng',
 'gồm',
 'công_an',
 'tỉnh',
 'công_an',
 'tp',
 'huế',
 'nhanh_chóng',
 'có_mặt',
 'hiện_trường',
 'triển_khai',
 'đồng_bộ',
 'biện_pháp',
 'nghiệp_vụ'

In [15]:
for i in range(len(tokens_list)):
    tokens_list[i] = ' '.join(tokens_list[i])
tokens_list[0]

'chiều 31/7 công_an tỉnh thừa thiên_huế thông_tin ban_đầu vụ nổ_súng cướp tiệm vàng chợ đông ba nằm đường trần_hưng_đạo tp huế tỉnh thừa thiên_huế thông_sài gòn giải_phóng khoảng h30 31/7 đối_tượng sử_dụng súng ak bất_ngờ xông tiệm vàng hoàng_đức thái_lợi phía chợ đông ba rồi nổ_súng chỉ_thiên liên_tiếp uy_hiếp chủ tiệm cướp vàng đối_tượng mang số vàng vừa cướp vứt vỉa_hè rồi đi bộ khu_vực cầu gia_hội cách khu_vực gây án khoảng m giám_đốc công_an tỉnh thừa thiên_huế lập_tức trực_tiếp chỉ_đạo lực_lượng chức_năng gồm công_an tỉnh công_an tp huế nhanh_chóng có_mặt hiện_trường triển_khai đồng_bộ biện_pháp nghiệp_vụ khoanh vùng ngăn người_dân đi hiện_trường hàng trăm tiểu_thương chợ đông ba người_dân gần cầu gia_hội yêu_cầu di_chuyển khỏi hiện_trường đóng_cửa nhà đề_phòng đạn_lạc tuy_nhiên thấy vàng ném đường đua nhau nhặt tạo cảnh nhốn_nháo cổng chợ do đây khu_vực trung_tâm tp huế đông dân nên đảm_bảo an_toàn người_dân lực_lượng công_an tìm cách hướng đối_tượng nhà lục_giác khu_vực công_vi

In [16]:
# delete variables due to lack of RAM
del raw
del raw_content
del clean_content
del word_count
del all_words
gc.collect()

0

# Extract feature

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
# Using Bag of Words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tokens_list)

In [18]:
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 28614384 stored elements and shape (184539, 503189)>

In [19]:
X[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [20]:
X[0].toarray().shape

(1, 503189)

In [21]:
# Using bigram Bag of Words
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                    token_pattern=r'\b\w+\b', min_df=1)
X_bigram = bigram_vectorizer.fit_transform(tokens_list)

In [22]:
X_bigram

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 72920954 stored elements and shape (184539, 9012648)>

In [23]:
# Using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(tokens_list)

In [24]:
X_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 28614384 stored elements and shape (184539, 503189)>

In [25]:
X_tfidf[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [26]:
X_tfidf[0].toarray().shape

(1, 503189)

In [27]:
# Using Hashing trick
hv = HashingVectorizer()
X_hv = hv.transform(tokens_list)

In [28]:
X_hv

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 28613144 stored elements and shape (184539, 1048576)>

In [29]:
X_hv[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])